In [131]:
import json
import yfinance as yf
import pandas as pd
import numpy as np
from pandas.tseries.offsets import BDay

In [62]:
amzn = yf.Ticker("AMZN")
amzn_hist = amzn.history(period="5y", auto_adjust=True)
amzn_hist = amzn_hist[['Close']]
amzn_hist = amzn_hist.rename(columns={"Close": "AMZN"})
amzn.history(period="5y", auto_adjust=True)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-04-20,899.700012,905.320007,896.289978,902.059998,2814400,0,0
2017-04-21,902.669983,903.650024,896.770020,898.530029,2420500,0,0
2017-04-24,908.679993,909.989990,903.820007,907.409973,3122900,0,0
2017-04-25,907.039978,909.479980,903.000000,907.619995,3380600,0,0
2017-04-26,910.299988,915.750000,907.559998,909.289978,2608900,0,0
...,...,...,...,...,...,...,...
2022-04-12,3073.850098,3101.979980,3007.659912,3015.750000,2758900,0,0
2022-04-13,3000.370117,3120.500000,2992.000000,3110.820068,2669500,0,0
2022-04-14,3107.800049,3117.939941,3029.439941,3034.129883,2575200,0,0


In [65]:
wmt = yf.Ticker("WMT")
wmt_hist = wmt.history(period="5y", auto_adjust=True)
wmt_hist = wmt_hist[['Close']]
wmt_hist = wmt_hist.rename(columns={"Close": "WMT"})
wmt.history(period="5y", auto_adjust=True)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-04-20,67.225835,68.105375,67.198633,67.824287,7681300,0.0,0
2017-04-21,67.769880,68.105377,67.642937,67.951233,5755700,0.0,0
2017-04-24,68.078155,68.368312,67.479702,67.806129,7812500,0.0,0
2017-04-25,67.978439,68.159784,67.878696,68.050980,5720400,0.0,0
2017-04-26,68.250433,68.703804,68.159761,68.395515,6948500,0.0,0
...,...,...,...,...,...,...,...
2022-04-12,154.240005,155.399994,152.550003,153.229996,8920400,0.0,0
2022-04-13,153.449997,157.789993,153.350006,157.220001,9761500,0.0,0
2022-04-14,157.110001,158.289993,156.429993,157.080002,7452300,0.0,0


In [71]:
hist = amzn_hist.merge(wmt_hist, how='inner', on='Date')
hist['log_ret_AMZN'] = np.log(hist['AMZN']) - np.log(hist['AMZN'].shift(1))
hist['log_ret_WMT'] = np.log(hist['WMT']) - np.log(hist['WMT'].shift(1))
hist

,AMZN,WMT,log_ret_AMZN,log_ret_WMT
Date,,,,
2017-04-20,902.059998,67.824280,NaN,NaN
2017-04-21,898.530029,67.951233,-0.003921,0.001870
2017-04-24,907.409973,67.806129,0.009834,-0.002138
2017-04-25,907.619995,68.050972,0.000231,0.003604
2017-04-26,909.289978,68.395531,0.001838,0.005050
...,...,...,...,...
2022-04-12,3015.750000,153.229996,-0.002216,-0.006894
2022-04-13,3110.820068,157.220001,0.031038,0.025706
2022-04-14,3034.129883,157.080002,-0.024962,-0.000891


In [78]:
corr_hist = hist['AMZN'].rolling(5).corr(hist['WMT'])
corr_hist = corr_hist.to_frame(name='corr_coef')
corr_hist

,corr_coef
Date,
2017-04-20,NaN
2017-04-21,NaN
2017-04-24,NaN
2017-04-25,NaN
2017-04-26,0.507221
...,...
2022-04-12,0.554899
2022-04-13,0.820890
2022-04-14,0.774786


# Get Position Direction

In [109]:
def get_position_direction(log_returns):
    amzn_neg_count = np.sum((log_returns['log_ret_AMZN'].values < 0))
    wmt_neg_count = np.sum((log_returns['log_ret_WMT'].values < 0))
    amzn_direction = 'LONG' if (amzn_neg_count >= wmt_neg_count) else 'SHORT'
    wmt_direction = 'SHORT' if (amzn_direction == 'LONG') else 'LONG'
    return {'AMZN': amzn_direction, 'WMT': wmt_direction}

# Define Trade

In [137]:
def define_trade(corr_hist):
    current_pos_status = json.load(open('position_status.json'))['status']
    print(current_pos_status)
    for index, row in corr_hist.iterrows():
        if row['corr_coef'] < 0 and current_pos_status == 'CLOSE':
            from_date = index - BDay(4)
            to_date = index
            slice_hist = hist.loc[from_date: to_date, "log_ret_AMZN":"log_ret_WMT"]
            print(slice_hist)
            trade = get_position_direction(slice_hist)
            trade['trade_date'] = index + BDay(1)
            return trade
print(define_trade(corr_hist))

CLOSE
            log_ret_AMZN  log_ret_WMT
Date                                 
2017-04-25      0.000231     0.003604
2017-04-26      0.001838     0.005050
2017-04-27      0.009947     0.000133
2017-04-28      0.007172    -0.003453
2017-05-01      0.024814     0.000665
{'AMZN': 'SHORT', 'WMT': 'LONG', 'trade_date': Timestamp('2017-05-02 00:00:00')}


# Open Position

In [116]:
trade = define_trade(corr_hist)

            log_ret_AMZN  log_ret_WMT
Date                                 
2017-04-25      0.000231     0.003604
2017-04-26      0.001838     0.005050
2017-04-27      0.009947     0.000133
2017-04-28      0.007172    -0.003453
2017-05-01      0.024814     0.000665
{'AMZN': 'SHORT', 'WMT': 'LONG', 'trade_date': Timestamp('2017-05-02 00:00:00')}
